# Downloads Mathur Sample and MAST Kepler Magnitudes for the Mathur Stars.

This code will take the Mathur sample and pull a KIC ID for each star. These are then split up into files with a length of no more than 999 (as the limit for a MAST file search is 10,000). 

These files can then be used in the MAST KIC search and retrieve form to pull the Kepler magnitudes for each star. 

When this is done these magnitude files can be combined with the temperature, logg and KIC of each star in the Mathur sample to create a file to cross-match with the TRILEGAL sample. 

In [2]:
# Necessary Modules
import numpy as np 
import pandas as pd
import glob

In [3]:
# Open the Mathur sample
mathur = pd.read_csv('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/mathur_2017_edited.txt', delimiter='|')
print('There are ' + str(len(mathur)) + ' in the Mathur sample.')

There are 197096 in the Mathur sample.


In [19]:
# Pulls just the KIC IDs from the Mathur sample and saved them into a new file
kic = list(mathur['KIC']) 
rows = zip(kic) 
header = ['kic']
df = pd.DataFrame(rows, columns=header)
df.to_csv('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/mathur_2017_kic_ids.csv', index=False)

In [10]:
no_of_files = round(len(mathur)/10000,0)
print('We will need: ' + str(no_of_files) + ' files of KIC IDs for the MAST search.')

We will need: 20.0 files of KIC IDs for the MAST search.


In [13]:
# Open list of KICs
kic_ids = pd.read_csv('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/mathur_2017_kic_ids.csv')
kic_ids = list(kic_ids['kic'])

In [14]:
# Break the KIC list into 20 lists, in order, and save them to individual files.
index_list = np.linspace(0, len(mathur), (int(no_of_files) + 1))
list_lengths = []

a = 1
for i in range(len(index_list)-1): 
    kic_list = []
    print('\nFirst index: ' + str(index_list[i]))
    print('Last index: ' + str(index_list[i+1]))
    kic_list = kic_ids[int(index_list[i]):int(index_list[i+1])]
    
    print('First KIC: ' + str(kic_list[0]))
    print('Last KIC: ' + str(kic_list[-1]))
    
    print('Length of KIC list: ' + str(len(kic_list)))
    list_lengths.append(len(kic_list))
    
    with open('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/kic_list_' + str(a) + '.txt', 'w') as output:
         for row in kic_list:
            output.write(str(row) + '\n')
    
    a = a + 1
    
print('\nThe total number of KIC IDs we need to search for is: ' + str(len(kic_ids)))
print('The combined number of KIC IDs in all these lists is: ' + str(np.sum(list_lengths)))


First index: 0.0
Last index: 9854.8
First KIC: 757076
Last KIC: 3337733
Length of KIC list: 9854

First index: 9854.8
Last index: 19709.6
First KIC: 3337740
Last KIC: 4067493
Length of KIC list: 9855

First index: 19709.6
Last index: 29564.399999999998
First KIC: 4067498
Last KIC: 4771739
Length of KIC list: 9855

First index: 29564.399999999998
Last index: 39419.2
First KIC: 4771770
Last KIC: 5270561
Length of KIC list: 9855

First index: 39419.2
Last index: 49274.0
First KIC: 5270627
Last KIC: 5707589
Length of KIC list: 9855

First index: 49274.0
Last index: 59128.799999999996
First KIC: 5707592
Last KIC: 6201205
Length of KIC list: 9854

First index: 59128.799999999996
Last index: 68983.59999999999
First KIC: 6201235
Last KIC: 6678316
Length of KIC list: 9855

First index: 68983.59999999999
Last index: 78838.4
First KIC: 6678322
Last KIC: 7188323
Length of KIC list: 9855

First index: 78838.4
Last index: 88693.2
First KIC: 7188366
Last KIC: 7676656
Length of KIC list: 9855

First 

At this point the user must use these KIC lists to download the necessary values from MAST... there is probably an API for this but it is not used here. 

In [15]:
# Open the new magnitude files. 
path = '/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/'
kic_lists = glob.glob(path + 'kic_list_*.txt')
searches = glob.glob(path + 'kic10_search-*.txt')

In [16]:
# Save the new magnitude files into one big file. 
length_of_searches = []
file_list = []

for i in searches: 
    print('File number: ' + str(i[58:-4]))
    file = pd.read_csv(i, skiprows=[1])
    file_list.append(file)
    length_of_searches.append(len(file))
    
df_list = file_list
full_df = pd.concat(df_list)
print('Individual length: ' + str(np.sum(length_of_searches)))
print('Complete length: ' + str(len(full_df)))

full_df.to_csv(path + 'mathur_magnitudes_complete.csv', index=False)

File number: 8
File number: 9
File number: 20
File number: 19
File number: 18
File number: 16
File number: 17
File number: 15
File number: 14
File number: 10
File number: 11
File number: 13
File number: 12
File number: 4
File number: 5
File number: 7
File number: 6
File number: 2
File number: 3
File number: 1
Individual length: 197096
Complete length: 197096


In [17]:
# Find the correct magnitude to correspond to each Kepler magnitude using the indexing of the Kepler ID. 
mathur = pd.read_csv('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/mathur_2017_edited.txt', delimiter='|')
mathur_kic = mathur['KIC']
mathur_teff = mathur['Teff']
mathur_logg = mathur['log(g)']

kepler = full_df
kepler_kic = list(kepler['Kepler ID'])
kepler_mag = list(kepler['Kepler Mag'])

indices = []
magnitudes = []
for i in mathur_kic:
     index = kepler_kic.index(i)
     indices.append(index)
     magnitudes.append(kepler_mag[index])

In [133]:
# Save into Mathur file with the parameters needed for the cross-match (with TRILEGAL).
rows = zip(mathur_kic, mathur_teff, mathur_logg, magnitudes)
header = ['kic', 'teff', 'logg', 'kep_mag']
df = pd.DataFrame(rows, columns=header)
df.to_csv('/Users/Jess/SF_m_dwarfs/SF_mathur_magnitudes/complete_mathur_file.csv', index=False)